In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w11-p1/Label2Names.csv
/kaggle/input/2021-ai-w11-p1/submission.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0080.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0015.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0066.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0049.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0063.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0020.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0028.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0014.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0054.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0004.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0009.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0021.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/image_0070.csv
/kaggle/input/2021-ai-w11-p1/train_csv_v2/scorpion/im

In [2]:
# 현재 opencv 코드는 3.4.2.17 에 맞춰서 작성되어 있어 버전을 맞추는 작업이 필요하다. 
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.17
! yes | pip3 install opencv-contrib-python==3.4.2.17

Found existing installation: opencv-python 4.5.4.58
Uninstalling opencv-python-4.5.4.58:
  Would remove:
    /opt/conda/lib/python3.7/site-packages/cv2/*
    /opt/conda/lib/python3.7/site-packages/opencv_python-4.5.4.58.dist-info/*
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Core-39545cc7.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Gui-ba0a2070.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Test-c38a5234.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Widgets-e69d94fb.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5XcbQpa-ca221f44.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libX11-xcb-69166bdf.so.1.0.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libavcodec-64ac49e1.so.58.91.100
    /opt/conda/lib/python3.7/si

In [3]:
import cv2
cv2.__version__

'3.4.2'

In [4]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch 



In [5]:
dataroot="/kaggle/input/2021-ai-w11-p1/"
csvpath=os.path.join(dataroot,"Label2Names.csv")
df_data=pd.read_csv(csvpath,header=None)
dataroottrain=os.path.join(dataroot,"train_csv_v2")
dataroottest=os.path.join(dataroot,"test_csv_v2")

# >> 필요한 모듈 부분 (function 부)

In [6]:
# -------------------------------------
# 학습데이터, 테스트 데이터 불러오기 
# -------------------------------------
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image


class trainDataset(Dataset):
    def __init__(self, dataroottrain):
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        self.trainlabel=[]
        self.trainpath=[]
        for classname in tqdm(os.listdir(dataroottrain)):
            ## class name to index
            if classname =="BACKGROUND_Google":
                labelindex=101
            else:
                
                
                labelindex=(df_data.index[df_data[1]==classname]).tolist()[0]
                
            for csvname in os.listdir(os.path.join(dataroottrain,classname)):

                self.trainlabel.append(labelindex)
                ## 데이터 경로 저장 
                csvpath=os.path.join(dataroottrain,classname,csvname)
                self.trainpath.append(csvpath)

    def __getitem__(self, idx):
        ## index 에 해당하는 label 값 과 영상 데이터 받아오기 
        csvpath=self.trainpath[idx]
        label=self.trainlabel[idx]
        ## 1D 데이터를 영상으로 변환 
        
        img=np.array(pd.read_csv(csvpath)).reshape((256,256,3))
        
        ## numpy 데이터를 tensor 형태로 변환 
        
        img=img.astype(np.uint8)
        img = Image.fromarray(img)
        ## numpy 데이터를 tensor 형태로 변환 
        img=self.transform(img)

        

        label=torch.tensor(label)
        return img, label
    
    def __len__(self):
        return len(self.trainpath)
    
class testDataset(Dataset):
    def __init__(self, dataroottest):
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        self.testlabel=[]
        self.testpath=[]
        testsort=sorted(os.listdir(dataroottest))
        for csvname in tqdm(testsort):
            ## 영상 경로 저장 
            csvpath=os.path.join(dataroottest,csvname)
            self.testpath.append(csvpath)
            self.testlabel.append(csvname)
            
    def __getitem__(self, idx):
        csvpath=self.testpath[idx]
        label=self.testlabel[idx]
        ## 1D 데이터를 영상으로 변환 
        img=np.array(pd.read_csv(csvpath)).reshape((256,256,3))
        img=img.astype(np.uint8)
        img = Image.fromarray(img)
        ## numpy 데이터를 tensor 형태로 변환 
        img=self.transform(img)

        return img,label
    
    def __len__(self):
        return len(self.testpath)



In [7]:
train_data = trainDataset(dataroottrain)
test_data = testDataset(dataroottest)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=16)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False, num_workers=4)

100%|██████████| 1712/1712 [00:00<00:00, 355724.19it/s]
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [8]:
device='cuda' if torch.cuda.is_available() else 'CPU'
print(device)

cuda


In [9]:
import torchvision.models as models
model = models.vgg19(pretrained=True)
num_classes=102
model.classifier[6] = torch.nn.Linear(4096,num_classes)

for param in model.features.parameters():
    param.requires_grad = False
    
model=model.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [10]:
for name, param in model.named_parameters():
    if param.requires_grad == True :
        print(name)

classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias
classifier.6.weight
classifier.6.bias


In [11]:
Loss=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3) 

In [12]:
model.train()
for epoch in range(100):
    correct_pred = 0
    epoch_loss = 0
    for X,Y in train_loader:
        X= X.to(device)
        Y= Y.to(device)

        optimizer.zero_grad()
        hypothesis=model(X)
        
        _, preds=torch.max(hypothesis,1)
        
        Cost=Loss(hypothesis,Y)
        Cost.backward()
        optimizer.step()
        correct_pred = correct_pred + torch.sum(preds==Y)
        epoch_loss = epoch_loss + Cost.item()*X.size(0)

    print("epoch : {:02d} , Accuracy : {:.4f} , Loss : {:.4f}".format(epoch+1 , correct_pred/len(train_loader.dataset),epoch_loss/len(train_loader.dataset)))

epoch : 01 , Accuracy : 0.2056 , Loss : 3.8540
epoch : 02 , Accuracy : 0.4663 , Loss : 2.9537
epoch : 03 , Accuracy : 0.5732 , Loss : 3.1264
epoch : 04 , Accuracy : 0.6239 , Loss : 3.2886
epoch : 05 , Accuracy : 0.6539 , Loss : 3.1332
epoch : 06 , Accuracy : 0.6997 , Loss : 2.7349
epoch : 07 , Accuracy : 0.7343 , Loss : 2.3363
epoch : 08 , Accuracy : 0.7693 , Loss : 2.3200
epoch : 09 , Accuracy : 0.7552 , Loss : 2.9029
epoch : 10 , Accuracy : 0.7748 , Loss : 2.5921
epoch : 11 , Accuracy : 0.7935 , Loss : 2.9591
epoch : 12 , Accuracy : 0.8167 , Loss : 2.5384
epoch : 13 , Accuracy : 0.8337 , Loss : 2.3357
epoch : 14 , Accuracy : 0.8222 , Loss : 3.0763
epoch : 15 , Accuracy : 0.8258 , Loss : 3.0381
epoch : 16 , Accuracy : 0.8330 , Loss : 3.4517
epoch : 17 , Accuracy : 0.8389 , Loss : 3.0608
epoch : 18 , Accuracy : 0.8690 , Loss : 2.7091
epoch : 19 , Accuracy : 0.8670 , Loss : 2.7593
epoch : 20 , Accuracy : 0.8480 , Loss : 3.9063
epoch : 21 , Accuracy : 0.8614 , Loss : 3.6239
epoch : 22 , 

In [13]:
prediction=list()
with torch.no_grad() : #test시 작성
    for X,_ in test_loader:
        X=X.to(device)
        hypothesis=model(X)
        _,preds=torch.max(hypothesis,1)
        prediction.extend((preds+1).tolist())

In [14]:
submit=pd.read_csv("/kaggle/input/2021-ai-w11-p1/submission.csv")
submit['Category']=prediction
submit.to_csv('submit.csv',index=False)